# Acey Deucey Probability Breakdown

## Creating the Probability Distribution

Let P(A) = Probability that a given bet wins <br>
Need to create a PMF since outcomes are discrete... <br>


$ P(A) = \frac{i-j-1}{13} $ where i,j are high and low card showing, respectively

This formula assumes replacement... which is not realistic, but a good starting point

### Considering Expected Value

Upon first glance, it seems that we should bet when $ P(A) > 50 $%... However, the catch to the game is that you can "post," and lose double your bet. So, instead of betting the probability, we must bet the expected value.

We will now define the return function... <br>
$ R(win) = x $ if i < k < j <br>
$ R(loss) = -x $ if k > i OR k < j <br>
$ R(post) = -2x $ if i < k < j <br>
Where x represents the value of the bet and k represents the value of the third card

### Creating a Counting Strategy to Create Better Distribution

Let's play a test round... <br>
1. Dealer deals a 3 and a King <br>
$ E(A) = \sum_{n=1}^{3}P(n)*R(n) = P(win)*R(win)+P(loss)*R(loss)+P(post)*P(post) $ <br>
$ E(A) = \frac{9}{13} * x + \frac{2}{13} * -x + \frac{2}{13}* -2x = \frac{5x}{13}$ <br>
We have a positive expected return, so we would choose to bet in this scenerion and expect a return of $ \frac{5}{13} $ of our bet. <br>
A. We bet 10, Dealer deals a 5... we win! Return is 10.
2. Dealer deals a 4 and a Queen <br>
$ E(A) = \sum_{n=1}^{3}P(n)*R(n) = P(win)*R(win)+P(loss)*R(loss)+P(post)*P(post) $ <br>
$ E(A) = \frac{7*4-1}{52-5} * x + \frac{4*4-2}{52-5} * -x + \frac{2*4}{52-5}* -2x = \frac{-3x}{47}$ <br>
Negative expected return... we don't bet. Let's look at what's happening here...

$ P(win) = \frac{Cards between i and j}{Cards Remaining} $
$ P(loss) = \frac{Cards outside i and j}{Cards Remaining} $
$ P(post) = \frac{Cards =i and j}{Cards Remaining} $

### Basic Strategy

$ P(win | spread) = \frac{spread-1}{13}$

$0.5 = \frac{spread-1}{13}$

$ 6.5 = spread - 1 $

Spread = 7.5 for P(win) = 0.5... therefore spread must be 8 to be greater that 50% odds without considering posting or non replacement in the deck.

Example 8 card spreads: Queen/4, King/5, High Ace/6, Jack/3, 10/2, 9/Low Ace

So basic strategy will be to bet 8 card spreads with a counting strategy...

### Counting Strategy

Let's use our middle-most betting hand (Queen/4) to define our first negative cards... <br>
Therefore, 5-Jack are all negative value cards. <br>
7,8 have NO chance of hurting us with our betting strategy, so let's define them as -2, and all others as -1.

Now let's look at our positive cards... <br>
Ace will be defined as +2, with 2-4 and Queen, King being +1

|Card|Value|
|---|---|
|Ace|+2|
|2|+1|
|3|+1|
|4|+1|
|5|-1|
|6|-1|
|7|-2|
|8|-2|
|9|-1|
|10|-1|
|Jack|-1|
|Queen|+1|
|King|+1|

# Simulation

In [187]:
from random import randrange
import pandas as pd

class Card:
    def __init__(self, value, color):
        self.value = value
        self.color = color

In [188]:
num_players = 10
min_bet = num_players
count = 0
correct_bets = []

In [234]:
def play_round(num_rounds, spread_to_bet, count_values, print_round=True):
    returns = 0
    correct_bets_round_num = 0
    correct_bets_round_denom = 0
    colors = ['Hearts', 'Diamonds', 'Spades', 'Clubs']
    values = ['Ace', 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
    deck = [Card(value, color) for value in values for color in colors]
    count = 0
    pot = 10
    for r in range(num_rounds):
        if print_round:
            print(f'______________________ROUND {r+1}______________________')
        last_cards = []
        for player in range(num_players):
            if len(deck) < 3:
                deck = [Card(value, color) for value in values for color in colors]
                if print_round:
                    print(f'New Deck!')
                count = 0
            left_card = deck.pop(randrange(len(deck)))
            right_card = deck.pop(randrange(len(deck)))
            if print_round:
                print(f'Player {player} is dealt: {left_card.value} of {left_card.color} , {right_card.value} of {right_card.color}')
            count += count_values[left_card.value]
            count += count_values[right_card.value]
            if left_card.value == 'Ace':
                left_card.value = 1
            if right_card.value == 'Ace':
                right_card.value = 14
            spread = abs(right_card.value - left_card.value)
            bet = 0
            outer_cards = [min(right_card.value,left_card.value), max(right_card.value,left_card.value)]
            if player == 1 and not bet_placed:
                if left_card.value <= last_cards[0] or left_card.value >= last_cards[1]:
                    correct_bets_round_num += 1
                    if print_round:
                        print('Good bet')
                correct_bets_round_denom += 1
            if player == 0:
                if spread >= spread_to_bet:
                    bet_placed = True
                    if count > 0:
                        if spread == 6:
                            ideal_bet = 10
                            bet = min(ideal_bet, pot)
                            deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns, print_round)
                        elif spread == 7:
                            ideal_bet = 10
                            bet = min(ideal_bet, pot)
                            deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns, print_round)
                        elif spread == 8:
                            ideal_bet = 20
                            bet = min(ideal_bet, pot)
                            deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns, print_round)
                        elif spread == 9:
                            ideal_bet = 30
                            bet = min(ideal_bet, pot)
                            deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns, print_round)
                        elif spread == 10:
                            ideal_bet = 40
                            bet = min(ideal_bet, pot)
                            deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns, print_round)
                        elif spread == 11:
                            ideal_bet = 40
                            bet = min(ideal_bet, pot)
                            deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns, print_round)
                        elif spread == 12:
                            ideal_bet = 50
                            bet = min(ideal_bet, pot)
                            deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns, print_round)
                        elif spread == 13:
                            bet = pot
                            deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns, print_round)
                else:
                    last_cards = outer_cards
                    bet_placed = False
            else:
                if spread == 6:
                    bet = 10
                    deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns, print_round)
                elif spread == 7:
                    ideal_bet = 20
                    bet = min(ideal_bet, pot)
                    deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns, print_round)
                elif spread == 8:
                    ideal_bet = 30
                    bet = min(ideal_bet, pot)
                    deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns, print_round)
                elif spread == 9:
                    ideal_bet = 30
                    bet = min(ideal_bet, pot)
                    deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns, print_round)
                elif spread == 10:
                    ideal_bet = 40
                    bet = min(ideal_bet, pot)
                    deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns, print_round)
                elif spread == 11:
                    ideal_bet = 50
                    bet = min(ideal_bet, pot)
                    deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns, print_round)
                elif spread == 12:
                    ideal_bet = 50
                    bet = min(ideal_bet, pot)
                    deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns, print_round)
                elif spread == 13:
                    bet = pot
                    deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns, print_round)
        if print_round:
            print(f'Count: {count}')
        returns -= 1
    if correct_bets_round_denom != 0:
        correct_bets.append(correct_bets_round_num / correct_bets_round_denom)
    return returns, correct_bets


def deal_middle_card(deck, outer_cards, count, pot, bet, player, returns, print_round):
    middle_card = deck.pop(randrange(len(deck)))
    if print_round:
        print(f'Middle Card: {middle_card.value} of {middle_card.color}')
    count += count_values[middle_card.value]
    if middle_card.value == 'Ace':
        if outer_cards[0] == 1:
            middle_card.value = 1
        elif outer_cards[1] == 14:
            middle_card.value = 14
        else:
            middle_card.value = 14
    if outer_cards[0] < middle_card.value and middle_card.value < outer_cards[1]:
        if player == 0:
            returns += bet
        pot -= bet
        if pot < 10:
            pot = 10
        if print_round:
            print(f'Bet Won! Player {player} wins ${bet}!')
    elif middle_card.value == outer_cards[0] or middle_card.value == outer_cards[1]:
        if player == 0:
            returns -= bet * 2
        pot += bet * 2
        if print_round:
            print(f'POST!!! Suck it player {player}')
    else:
        if player == 0:
            returns -= bet
        pot += bet
    if player == 0:
        if print_round:
            print(f'Total Returns: {returns}')
    if print_round:
        print(f'Pot Value: ${pot}')
        
    return deck, count, pot, returns

In [254]:
def run_simulation(num_trials, num_rounds, counting_strategies, spreads):
    k = 0
    strat_num = len(counting_strategies)
    column_names = []
    max_returns = 0
    optimal = [0,0]
    for i in range(strat_num):
        column_names.append(f'Counting Strategy {i+1}')
    df_list_returns = []
    df_list_bets = []
    for spread in spreads:
        spread_list_returns = []
        spread_list_bets = []
        for count_key, count in counting_strategies.items():
            returns = []
            correct_bets = []
            for trial in range(num_trials):
                if k == 0:
                    print_round = True
                    print(f'------------------------------------------------------TRIAL {trial+1}------------------------------------------------------')
                else:
                    print_round = False
                winnings = play_round(num_rounds, spread, count, print_round)[0]
                corr_bets = play_round(num_rounds, spread, count, print_round)[1]
                correct_bets.append(corr_bets)
                returns.append(winnings)
                k += 1
            corr_bets_avg = sum(corr_bets) / len(corr_bets) * 100
            returns_avg = sum(returns) / len(returns)
            if returns_avg > max_returns:
                max_returns = returns_avg
                optimal = [spread, count_key]
            spread_list_returns.append(returns_avg)
            spread_list_bets.append(f'{round(corr_bets_avg, 2)}%')
        df_list_returns.append(spread_list_returns)
        df_list_bets.append(spread_list_bets)
    df_returns = pd.DataFrame(df_list_returns, index=spreads, columns=column_names)
    df_bets = pd.DataFrame(df_list_bets, index=spreads, columns=column_names)
    
    return df_returns, df_bets, optimal, max_returns



In [279]:
spreads_to_test = [4,5,6,7,8,9,10,11]
strat_to_test = {1:{'Ace': 2, 2:1, 3:1, 4:1, 5:-1, 6:-1, 7:-2, 8:-2, 9:-1, 10:-1, 11:-1, 12:1, 13:1},
                 2:{'Ace': 2, 2:1, 3:1, 4:1, 5:-1, 6:-1, 7:-1, 8:-1, 9:-1, 10:-1, 11:-1, 12:1, 13:1},
                 3:{'Ace': 2, 2:1, 3:1, 4:1, 5:-1, 6:-1, 7:-1, 8:-1, 9:-1, 10:1, 11:1, 12:1, 13:1},
                 4:{'Ace': 2, 2:1, 3:1, 4:1, 5:1, 6:-1, 7:-1, 8:-1, 9:-1, 10:-1, 11:-1, 12:1, 13:1}}

df_returns, df_bets, optimal, max_returns = run_simulation(100, 30, strat_to_test, spreads_to_test)

------------------------------------------------------TRIAL 1------------------------------------------------------
______________________ROUND 1______________________
Player 0 is dealt: 13 of Spades , 2 of Spades
Middle Card: Ace of Diamonds
Total Returns: -10
Pot Value: $20
Player 1 is dealt: Ace of Clubs , 5 of Hearts
Player 2 is dealt: 5 of Diamonds , 3 of Spades
Player 3 is dealt: 9 of Spades , 12 of Hearts
Player 4 is dealt: 5 of Spades , 12 of Diamonds
Middle Card: 6 of Spades
Bet Won! Player 4 wins $20!
Pot Value: $10
Player 5 is dealt: 5 of Clubs , Ace of Spades
Middle Card: 8 of Diamonds
Bet Won! Player 5 wins $10!
Pot Value: $10
Player 6 is dealt: 3 of Diamonds , 8 of Spades
Player 7 is dealt: 9 of Hearts , 13 of Clubs
Player 8 is dealt: 10 of Hearts , 8 of Clubs
Player 9 is dealt: 2 of Hearts , Ace of Hearts
Middle Card: 7 of Spades
Bet Won! Player 9 wins $10!
Pot Value: $10
Count: 2
______________________ROUND 2______________________
Player 0 is dealt: 11 of Spades , 10 of

In [280]:
print(f'Most Returns: ${max_returns}')
print('Returns for Spread, Counting Strategy (as defined above)')
df_returns

Most Returns: $18.0
Returns for Spread, Counting Strategy (as defined above)


,Counting Strategy 1,Counting Strategy 2,Counting Strategy 3,Counting Strategy 4
4,-17.4,-16.3,-15.8,-1.1
5,-4.2,-1.8,3.5,-4.4
6,-13.0,-0.4,0.9,-3.5
7,-14.2,-3.3,-2.2,1.8
8,-15.8,-9.5,18.0,12.1
9,-8.7,8.6,12.4,5.2
10,-17.2,-4.5,4.2,3.3
11,-14.0,-15.9,-13.1,-8.7


In [281]:
print('Percentage of Correct Non-Bets:')
df_bets

Percentage of Correct Non-Bets:


,Counting Strategy 1,Counting Strategy 2,Counting Strategy 3,Counting Strategy 4
4,82.87%,82.88%,82.89%,82.91%
5,82.92%,82.92%,82.93%,82.94%
6,82.95%,82.95%,82.95%,82.96%
7,82.96%,82.96%,82.96%,82.96%
8,82.96%,82.96%,82.96%,82.95%
9,82.95%,82.94%,82.94%,82.93%
10,82.92%,82.92%,82.91%,82.9%
11,82.89%,82.88%,82.87%,82.86%


In [282]:
print(f'Bet only spreads of {optimal[0]} or greater using the counting strategy:')
print(f'{strat_to_test[optimal[1]]} (Strategy #{optimal[1]})')

Bet only spreads of 8 or greater using the counting strategy:
{'Ace': 2, 2: 1, 3: 1, 4: 1, 5: -1, 6: -1, 7: -1, 8: -1, 9: -1, 10: 1, 11: 1, 12: 1, 13: 1} (Strategy #3)
